## 分类任务微调全流程

首先配置镜像

In [1]:
import os
# 设置hf-mirror镜像地址
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
# 查看是否成功结果
hf_endpoint = os.getenv('HF_ENDPOINT')
print('HF_ENDPOINT:', hf_endpoint)

HF_ENDPOINT: https://hf-mirror.com


构建hugging face数据集格式

In [4]:
from datasets import load_dataset
data_files = {"train":"./b站弹幕情感分析/data/train.tsv", "test":"./b站弹幕情感分析/data/test.tsv"}
ChnSetiCorp_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")
ChnSetiCorp_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text_a'],
        num_rows: 9146
    })
    test: Dataset({
        features: ['label', 'text_a'],
        num_rows: 1200
    })
})

加载模型评估metrics

In [5]:
import evaluate
metric = evaluate.combine(["accuracy", "f1", "precision", "recall", "matthews_correlation"])

加载模型以及tokenizer

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone-chinese")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone-chinese", num_labels=2, ignore_mismatched_sizes=True) 
#num_labels是输出的类别数量

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yiyanghkust/finbert-tone-chinese and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

整个数据集的tokenizer

In [13]:
def preprocess_function(example):
    return tokenizer(example["text_a"], truncation=True, max_length=128)

In [14]:
encoded_dataset = ChnSetiCorp_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/9146 [00:00<?, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [15]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text_a', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9146
    })
    test: Dataset({
        features: ['label', 'text_a', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1200
    })
})

参数初始化

In [11]:
from transformers import TrainingArguments, Trainer
model_name = "wosuibianzhaode"
task = "sentiment"
batch_size = 32
metric_name = "matthews_correlation"
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    report_to="none"
    # push_to_hub=True,
)

计算metrics的函数

In [16]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

训练

In [17]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipykernel_2896/1443369649.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Matthews Correlation
1,No log,0.236009,0.920000,0.918506,0.949123,0.889803,0.841794
2,0.224600,0.199986,0.934167,0.933108,0.961606,0.906250,0.869892
3,0.224600,0.236645,0.940833,0.939779,0.970228,0.911184,0.883422
4,0.084300,0.232808,0.945000,0.945634,0.947195,0.944079,0.889990
5,0.084300,0.241988,0.945833,0.945878,0.957841,0.934211,0.891962


TrainOutput(global_step=1430, training_loss=0.12051725587644777, metrics={'train_runtime': 237.097, 'train_samples_per_second': 192.875, 'train_steps_per_second': 6.031, 'total_flos': 3008017140403200.0, 'train_loss': 0.12051725587644777, 'epoch': 5.0})

模型评估（会根据效果最好的模型结果展示评估结果）

In [19]:
trainer.evaluate()

{'eval_loss': 0.24198751151561737,
 'eval_accuracy': 0.9458333333333333,
 'eval_f1': 0.9458784346378019,
 'eval_precision': 0.9578414839797639,
 'eval_recall': 0.9342105263157895,
 'eval_matthews_correlation': 0.8919622109294806,
 'eval_runtime': 2.0145,
 'eval_samples_per_second': 595.69,
 'eval_steps_per_second': 18.864,
 'epoch': 5.0}